In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
os.chdir("D:\Projects\LEGO\Rebrickable")
price = pd.read_csv("price.csv")
price.head()

,name,price,set_ID,rating,price_USD,year
0,2 X 8 Bricks,"9,$0.50 ,5.6c,Box,Retail,Normal","""215""",4.0,0.5,1961
1,4 x 4 Corner Bricks,"10,$0.50 ,5.0c,Box,Retail,Normal","""217""",4.0,0.5,1961
2,2 x 4 Bricks,"20,$0.50 ,2.5c,Box,Retail,Normal","""218""",4.0,0.5,1961
3,2 X 3 Bricks,"24,$0.50 ,2.1c,Box,Retail,Normal","""219""",4.0,0.5,1961
4,2 X 2 Bricks,"33,$0.50 ,1.5c,Box,Retail,Normal","""220""",4.0,0.5,1961


In [10]:
list(price[price.year == 2009].set_ID)

['"3594"',
 '"3595"',
 '"3596"',
 '"3597"',
 '"3833"',
 '"3834"',
 '"3835"',
 '"3836"',
 '"3837"',
 '"3838"',
 '"3839"',
 '"3840"',
 '"3841"',
 '"3843"',
 '"3844"',
 '"5486"',
 '"5487"',
 '"5488"',
 '"5489"',
 '"5538"',
 '"5539"',
 '"5545"',
 '"5546"',
 '"5547"',
 '"5560"',
 '"5587"',
 '"5592"',
 '"5594"',
 '"5595"',
 '"5632"',
 '"5633"',
 '"5634"',
 '"5635"',
 '"5636"',
 '"5637"',
 '"5638"',
 '"5639"',
 '"5640"',
 '"5641"',
 '"5969"',
 '"5970"',
 '"5971"',
 '"5972"',
 '"5973"',
 '"5974"',
 '"5979"',
 '"5980"',
 '"6191"',
 '"6192"',
 '"6193"',
 '"6194"',
 '"6239"',
 '"6240"',
 '"6241"',
 '"6242"',
 '"6243"',
 '"6253"',
 '"6741"',
 '"6742"',
 '"6743"',
 '"6745"',
 '"6747"',
 '"6751"',
 '"6752"',
 '"6753"',
 '"6754"',
 '"7078"',
 '"7079"',
 '"7097"',
 '"7195"',
 '"7196"',
 '"7197"',
 '"7198"',
 '"7199"',
 '"7630"',
 '"7631"',
 '"7632"',
 '"7633"',
 '"7634"',
 '"7635"',
 '"7636"',
 '"7637"',
 '"7638"',
 '"7639"',
 '"7641"',
 '"7642"',
 '"7682"',
 '"7683"',
 '"7685"',
 '"7686"',
 '"7687"',

### Adjust price with inflation

In [7]:
CPI = pd.read_csv("CPI.csv")
CPI.DATE = pd.to_datetime(CPI.DATE)
CPI["year"] = CPI.DATE.apply(lambda d: d.year)
CPI_year = CPI.groupby("year",as_index=False).agg("mean")
base = float(CPI_year[CPI_year.year == 2019].CPIAUCSL)
CPI_year["Index_19"] = CPI_year.CPIAUCSL.apply(lambda i: base/i)
CPI_year.head()

,year,CPIAUCSL,Index_19
0,1947,22.331667,11.374404
1,1948,24.045000,10.563918
2,1949,23.809167,10.668555
3,1950,24.062500,10.556235
4,1951,25.973333,9.779623


In [8]:
# combine price with CPI
price = price.merge(CPI_year.drop("CPIAUCSL",axis=1),on="year",how="left")

In [9]:
price["price_adjust"] = price.price_USD * price.Index_19
price["price_adjust"] = price["price_adjust"].apply(lambda i: round(i,2))
price.head()

,name,price,set_ID,rating,price_USD,year,Index_19,price_adjust
0,2 X 8 Bricks,"9,$0.50 ,5.6c,Box,Retail,Normal","""215""",4.0,0.5,1961,8.494824,4.25
1,4 x 4 Corner Bricks,"10,$0.50 ,5.0c,Box,Retail,Normal","""217""",4.0,0.5,1961,8.494824,4.25
2,2 x 4 Bricks,"20,$0.50 ,2.5c,Box,Retail,Normal","""218""",4.0,0.5,1961,8.494824,4.25
3,2 X 3 Bricks,"24,$0.50 ,2.1c,Box,Retail,Normal","""219""",4.0,0.5,1961,8.494824,4.25
4,2 X 2 Bricks,"33,$0.50 ,1.5c,Box,Retail,Normal","""220""",4.0,0.5,1961,8.494824,4.25


### Pieces data

In [11]:
os.chdir("D:\Projects\LEGO\Pieces")
files = os.listdir()
# transfer into dataframes
df_lst = [pd.read_csv(file) for file in files]
filenames = [file.split(".")[0] for file in files]
df_dic = dict(zip(filenames,df_lst))

In [12]:
# clean datasets with the same structure
def clean_data(file):
    df = df_dic[file]
    
    # create a new column called year
    df['year'] = int(file)
    
    # delete ":" in set_ID
    df['set_ID'] = df['set_ID'].str.replace(":","")
    df['set_ID'] = df['set_ID'].str.strip()
    
    return df

In [13]:
# iterate over all dataframes
df_clean = [clean_data(file) for file in filenames]
pcs = pd.concat(df_clean)
print(pcs.shape)

(15698, 3)


In [14]:
pcs = pcs.dropna()

In [15]:
pcs.set_ID = pcs.set_ID.apply('"{}"'.format)

In [16]:
list(pcs[pcs.year == 2009].set_ID)

['"3594"',
 '"3595"',
 '"3596"',
 '"3597"',
 '"3833"',
 '"3834"',
 '"3835"',
 '"3836"',
 '"3837"',
 '"3838"',
 '"3839"',
 '"3840"',
 '"3841"',
 '"3842"',
 '"3843"',
 '"3844"',
 '"4899"',
 '"5486"',
 '"5487"',
 '"5488"',
 '"5489"',
 '"5538"',
 '"5539"',
 '"5545"',
 '"5546"',
 '"5547"',
 '"5560"',
 '"5586"',
 '"5587"',
 '"5588"',
 '"5589"',
 '"5592"',
 '"5594"',
 '"5595"',
 '"5632"',
 '"5633"',
 '"5634"',
 '"5635"',
 '"5636"',
 '"5637"',
 '"5638"',
 '"5639"',
 '"5640"',
 '"5641"',
 '"5642"',
 '"5969"',
 '"5970"',
 '"5971"',
 '"5972"',
 '"5973"',
 '"5974"',
 '"5979"',
 '"5980"',
 '"6191"',
 '"6192"',
 '"6193"',
 '"6194"',
 '"6239"',
 '"6240"',
 '"6241"',
 '"6242"',
 '"6243"',
 '"6253"',
 '"6299"',
 '"6741"',
 '"6742"',
 '"6743"',
 '"6745"',
 '"6747"',
 '"6751"',
 '"6752"',
 '"6753"',
 '"6754"',
 '"7078"',
 '"7079"',
 '"7097"',
 '"7195"',
 '"7196"',
 '"7197"',
 '"7198"',
 '"7199"',
 '"7615"',
 '"7616"',
 '"7630"',
 '"7631"',
 '"7632"',
 '"7633"',
 '"7634"',
 '"7635"',
 '"7636"',
 '"7637"',

### merge piece with price

In [17]:
pcs.head()

,piece,set_ID,year
0,1,"""215""",1961
1,1,"""216""",1961
2,1,"""217""",1961
3,2,"""218""",1961
4,4,"""219""",1961


In [18]:
price.head()

,name,price,set_ID,rating,price_USD,year,Index_19,price_adjust
0,2 X 8 Bricks,"9,$0.50 ,5.6c,Box,Retail,Normal","""215""",4.0,0.5,1961,8.494824,4.25
1,4 x 4 Corner Bricks,"10,$0.50 ,5.0c,Box,Retail,Normal","""217""",4.0,0.5,1961,8.494824,4.25
2,2 x 4 Bricks,"20,$0.50 ,2.5c,Box,Retail,Normal","""218""",4.0,0.5,1961,8.494824,4.25
3,2 X 3 Bricks,"24,$0.50 ,2.1c,Box,Retail,Normal","""219""",4.0,0.5,1961,8.494824,4.25
4,2 X 2 Bricks,"33,$0.50 ,1.5c,Box,Retail,Normal","""220""",4.0,0.5,1961,8.494824,4.25


In [20]:
price_all = price.merge(pcs,how="left",on=["set_ID","year"])

In [25]:
price_all["pcs_num"] = price_all["price"].apply(lambda i: i.split(",")[0])

In [27]:
def adjust_pcs(item):
    if ("$" in item) or ("€" in item):
        item = np.nan
    return item

In [28]:
price_all.pcs_num = price_all.pcs_num.apply(adjust_pcs)

In [30]:
price_all.pcs_num = price_all.pcs_num.fillna(price_all.piece)

In [40]:
price_all[price_all.year == 2009]

,name,price,set_ID,rating,price_USD,year,Index_19,price_adjust,piece,pcs_num
3833,Bob's Workshop,"$14.99 | ,107.1c,USA: May 09, UK/EU: Mar 09,No...","""3594""",5.0,14.99,2009,1.183836,17.75,14,14
3834,Scoop at Bobland Bay,"$19.99 | ,71.4c,USA: May 09, UK/EU: Mar 09,Normal","""3595""",5.0,19.99,2009,1.183836,23.66,28,28
3835,Muck Can Do It,"$29.99 | ,76.9c,USA: Jun 09, UK/EU: Mar 09,Normal","""3596""",5.0,29.99,2009,1.183836,35.50,39,39
3836,Lofty and Dizzy Hard At Work,"$39.99 | ,78.4c,USA: May 09, UK/EU: Mar 09,Normal","""3597""",5.0,39.99,2009,1.183836,47.34,51,51
3837,Krusty Krab Adventures,"$24.99 | ,12.0c,Box,Retail,USA: Dec 08, UK/EU:...","""3833""",5.0,24.99,2009,1.183836,29.58,209,209
3838,Good Neighbours at Bikini Bottom,"$39.99 | ,9.4c,Box,Retail,USA: Dec 08, UK/EU: ...","""3834""",5.0,39.99,2009,1.183836,47.34,425,425
3839,Robo Champ,"$4.99 | ,4.2c,Box,Retail,USA: Jun 10, UK/EU: J...","""3835""",5.0,4.99,2009,1.183836,5.91,118,118
3840,Magikus,"$9.99 | ,9.3c,Box,Retail,USA: Jun 10, UK/EU: J...","""3836""",5.0,9.99,2009,1.183836,11.83,108,108
3841,Monster 4,"$14.99 | ,10.6c,Box,Retail,USA: Mar 10, UK/EU:...","""3837""",5.0,14.99,2009,1.183836,17.75,141,141
3842,Lava Dragon,"$14.99 | ,11.4c,Box,Retail,USA: Mar 10, UK/EU:...","""3838""",5.0,14.99,2009,1.183836,17.75,131,131


In [38]:
price_all.pcs_num = price_all.pcs_num.apply(lambda i: np.nan if i == "More" else i)

In [41]:
price_all.pcs_num = pd.to_numeric(price_all.pcs_num)

In [42]:
price_all = price_all.drop("piece",axis=1)

In [43]:
price_all["price_per_pcs"] = price_all.price_adjust/price_all.pcs_num

In [44]:
price_all["price_per_pcs"] = price_all["price_per_pcs"].apply(lambda i: round(i,2))
price_all.head()

,name,price,set_ID,rating,price_USD,year,Index_19,price_adjust,pcs_num,price_per_pcs
0,2 X 8 Bricks,"9,$0.50 ,5.6c,Box,Retail,Normal","""215""",4.0,0.5,1961,8.494824,4.25,9.0,0.47
1,4 x 4 Corner Bricks,"10,$0.50 ,5.0c,Box,Retail,Normal","""217""",4.0,0.5,1961,8.494824,4.25,10.0,0.42
2,2 x 4 Bricks,"20,$0.50 ,2.5c,Box,Retail,Normal","""218""",4.0,0.5,1961,8.494824,4.25,20.0,0.21
3,2 X 3 Bricks,"24,$0.50 ,2.1c,Box,Retail,Normal","""219""",4.0,0.5,1961,8.494824,4.25,24.0,0.18
4,2 X 2 Bricks,"33,$0.50 ,1.5c,Box,Retail,Normal","""220""",4.0,0.5,1961,8.494824,4.25,33.0,0.13


In [45]:
price_all["ppp_nominal"] = price_all.price_USD/price_all.pcs_num
price_all["ppp_nominal"] = price_all["ppp_nominal"].apply(lambda i: round(i,2))

In [46]:
price_all.head()

,name,price,set_ID,rating,price_USD,year,Index_19,price_adjust,pcs_num,price_per_pcs,ppp_nominal
0,2 X 8 Bricks,"9,$0.50 ,5.6c,Box,Retail,Normal","""215""",4.0,0.5,1961,8.494824,4.25,9.0,0.47,0.06
1,4 x 4 Corner Bricks,"10,$0.50 ,5.0c,Box,Retail,Normal","""217""",4.0,0.5,1961,8.494824,4.25,10.0,0.42,0.05
2,2 x 4 Bricks,"20,$0.50 ,2.5c,Box,Retail,Normal","""218""",4.0,0.5,1961,8.494824,4.25,20.0,0.21,0.03
3,2 X 3 Bricks,"24,$0.50 ,2.1c,Box,Retail,Normal","""219""",4.0,0.5,1961,8.494824,4.25,24.0,0.18,0.02
4,2 X 2 Bricks,"33,$0.50 ,1.5c,Box,Retail,Normal","""220""",4.0,0.5,1961,8.494824,4.25,33.0,0.13,0.02


In [47]:
price_all.isna().sum()

name                0
price               0
set_ID              0
rating            190
price_USD           0
year                0
Index_19            0
price_adjust        0
pcs_num          1477
price_per_pcs    1477
ppp_nominal      1477
dtype: int64

In [48]:
price_all[price_all.year == 2009]

,name,price,set_ID,rating,price_USD,year,Index_19,price_adjust,pcs_num,price_per_pcs,ppp_nominal
3833,Bob's Workshop,"$14.99 | ,107.1c,USA: May 09, UK/EU: Mar 09,No...","""3594""",5.0,14.99,2009,1.183836,17.75,14.0,1.27,1.07
3834,Scoop at Bobland Bay,"$19.99 | ,71.4c,USA: May 09, UK/EU: Mar 09,Normal","""3595""",5.0,19.99,2009,1.183836,23.66,28.0,0.84,0.71
3835,Muck Can Do It,"$29.99 | ,76.9c,USA: Jun 09, UK/EU: Mar 09,Normal","""3596""",5.0,29.99,2009,1.183836,35.50,39.0,0.91,0.77
3836,Lofty and Dizzy Hard At Work,"$39.99 | ,78.4c,USA: May 09, UK/EU: Mar 09,Normal","""3597""",5.0,39.99,2009,1.183836,47.34,51.0,0.93,0.78
3837,Krusty Krab Adventures,"$24.99 | ,12.0c,Box,Retail,USA: Dec 08, UK/EU:...","""3833""",5.0,24.99,2009,1.183836,29.58,209.0,0.14,0.12
3838,Good Neighbours at Bikini Bottom,"$39.99 | ,9.4c,Box,Retail,USA: Dec 08, UK/EU: ...","""3834""",5.0,39.99,2009,1.183836,47.34,425.0,0.11,0.09
3839,Robo Champ,"$4.99 | ,4.2c,Box,Retail,USA: Jun 10, UK/EU: J...","""3835""",5.0,4.99,2009,1.183836,5.91,118.0,0.05,0.04
3840,Magikus,"$9.99 | ,9.3c,Box,Retail,USA: Jun 10, UK/EU: J...","""3836""",5.0,9.99,2009,1.183836,11.83,108.0,0.11,0.09
3841,Monster 4,"$14.99 | ,10.6c,Box,Retail,USA: Mar 10, UK/EU:...","""3837""",5.0,14.99,2009,1.183836,17.75,141.0,0.13,0.11
3842,Lava Dragon,"$14.99 | ,11.4c,Box,Retail,USA: Mar 10, UK/EU:...","""3838""",5.0,14.99,2009,1.183836,17.75,131.0,0.14,0.11


In [49]:
price_all.to_csv("price_adj.csv",index=False)